This example shows how approximate distributions may not be unique when a system is in equilibrium. 

Note that this is a consequence of the model and not a limitation of the method because a random processes described in a model may have the same equlibrium distribution as that of another random process. Hence, we cannot uniquely determine a stochastic model from a given results set that only samples the system in equilibrium. Rather, we can only test whether a given model produces such a given results set. 

Note also that perturbed states seem to better demonstrate uniqueness, however such an approach is still insufficient to determine the model that produced a given results set. This naturally follows from the Taylor expansion. 

Probably the point to make is that our goal is not to provide a method for determining the model that produced a given results set. Rather, our goal is to test whether a given model produces a given dataset, and to make such tests possible at scale by providing guidelines, methods, tools, etc. that lead to reporting of necessary and sufficient data to perform such tests as common scientific practice. 

In [ ]:
import ipywidgets as ipw
from IPython.display import display

from stochastic_repro import start_pool
from stochastic_models import model_oscillator
from stochastic_tests import Test

t_fin = 10.0
num_steps = 100

test = Test(model=model_oscillator({'t0': ('norm', (0, 1E0))}),
            t_fin=t_fin,
            num_steps=num_steps,
            sample_times=[t_fin / num_steps * i for i in range(1, num_steps + 1)],
            trials=[10, 100, 1000, 10000],
            stochastic=False)

In [ ]:
test.execute_deterministic()
_ = test.plot_results_deterministic()

In [ ]:
label, out = ipw.Label(), ipw.Output()
display(out)
with out:
    display(label)

start_pool()
test.execute_stochastic(label)
_ = out.clear_output()

In [ ]:
_ = test.plot_results_stochastic()

In [ ]:
_ = test.plot_stats()

In [ ]:
_ = test.plot_distributions()

Note here that as the results sets converge to reliable distributions, those distributions are (practically) the same at equilibrium. 

In [ ]:
test.find_ecfs()
test.measure_ecf_diffs()

In [ ]:
for t in test.trials:
    ecf_ks_stat_t = {n: -1 for n in test.model.results_names}
    for ks in test.ecf_ks_stat[t]:
        for n, ks_stat in ks.items():
            ecf_ks_stat_t[n] = max(ecf_ks_stat_t[n], ks_stat)
    print(f'{t} trials')
    for n, ks_stat in ecf_ks_stat_t.items():
        print(f'\t{n}: {ks_stat}')

In [ ]:
preview_time = test.max_ks_stat_time(test.trials[-1])
# preview_time = test.min_final_eval_time(test.trials[-1])

test.plot_ecf(time=preview_time)
test.plot_ecf_diffs()
_ = test.plot_ecf_comparison(time=preview_time)

In [ ]:
test.generate_ecf_diff_fits()

In [ ]:
_, axs = test.plot_ecf_diff_fits(test.plot_ecf_diffs())
_ = axs[0].legend()

In [ ]:
# Runtime: ~3 minutes (M1 max)
test.test_sampling(err_thresh=1E-3)

In [ ]:
_ = test.plot_ks_sampling()

In [ ]:
test.generate_ecf_sampling_fits()
_, ax = test.plot_ecf_sampling_fits(test.plot_ecf_sampling())
_ = ax.legend()